In [3]:
import pulp

# Parameters (example values, replace with actual data)
n = 5  # Number of potential tour packages
m = 4  # Number of seasons
c_i = [10, 20, 15, 25, 30]  # Cost of designing and operating each package
c_k = [5, 10]  # Cost per unit of resource
p_i = [50, 60, 70, 80, 90]  # Price of each package
b_r = 1000  # Maximum budget for resources
R_k = [200, 300]  # Available amount of each resource
r_ijk = [[[5, 2], [3, 1], [4, 2], [2, 1]],  # Resource allocation for each package in each season
         [[6, 3], [4, 2], [5, 3], [3, 2]],
         [[7, 4], [5, 3], [6, 4], [4, 3]],
         [[8, 5], [6, 4], [7, 5], [5, 4]],
         [[9, 6], [7, 5], [8, 6], [6, 5]]]
d_min = [20, 30, 25, 35, 40]  # Minimum desired demand for each package
d_max = [50, 60, 55, 65, 70]  # Maximum desired demand for each package
d_ij = [[30, 40, 35, 45],  # Demand for each package in each season
        [40, 50, 45, 55],
        [35, 45, 40, 50],
        [45, 55, 50, 60],
        [50, 60, 55, 65]]
m_ij = [2, 3, 3, 2, 2]  # Maximum number of packages to offer per season

# Decision Variables
x = pulp.LpVariable.dicts("x", ((i, j) for i in range(n) for j in range(m)), cat='Binary')

# Objective Function
objective = pulp.lpSum(p_i[i] * d_ij[i][j] * x[i, j] for i in range(n) for j in range(m)) - \
            pulp.lpSum(c_i[i] * x[i, j] for i in range(n) for j in range(m)) - \
            pulp.lpSum(c_k[k] * r_ijk[i][j][k] * x[i, j] for i in range(n) for j in range(m) for k in range(len(c_k)))

# Create the problem
prob = pulp.LpProblem("TourPackageOptimization", pulp.LpMaximize)
prob += objective

# Constraints

# Budget Constraints
for i in range(n):
    for j in range(m):
        prob += pulp.lpSum(c_k[k] * r_ijk[i][j][k] * x[i, j] for k in range(len(c_k))) <= b_r

# Resource Constraints
for k in range(len(c_k)):
    for j in range(m):
        prob += pulp.lpSum(r_ijk[i][j][k] * x[i, j] for i in range(n)) <= R_k[k]

# Demand Constraints
for i in range(n):
    for j in range(m):
        prob += d_ij[i][j] * x[i, j] >= d_min[i]
        prob += d_ij[i][j] * x[i, j] <= d_max[i]

# Package Offering Constraints
for j in range(m):
    prob += pulp.lpSum(x[i, j] for i in range(n)) <= m_ij[j]

# Solve the problem
prob.solve()

# Print the results
for i in range(n):
    for j in range(m):
        print(f"Package {i} in season {j} ")

print(f"Objective value:$ {int(pulp.value(prob.objective))}")


Package 0 in season 0 
Package 0 in season 1 
Package 0 in season 2 
Package 0 in season 3 
Package 1 in season 0 
Package 1 in season 1 
Package 1 in season 2 
Package 1 in season 3 
Package 2 in season 0 
Package 2 in season 1 
Package 2 in season 2 
Package 2 in season 3 
Package 3 in season 0 
Package 3 in season 1 
Package 3 in season 2 
Package 3 in season 3 
Package 4 in season 0 
Package 4 in season 1 
Package 4 in season 2 
Package 4 in season 3 
Objective value:$ 35593


In [40]:
import pulp
import pandas as pd
import locale

# Set locale for currency formatting
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Load the Excel file
file_path = 'C:/Users/Mohammed/OneDrive/Desktop/use data.xlsx'
data = pd.read_excel(file_path, sheet_name='Data')

# Extract unique tour packages and seasons
tour_packages = data['Tour Name'].unique()
seasons = data['Season'].unique()

# Create integer indices for tour packages and seasons
tour_package_indices = {tour: idx for idx, tour in enumerate(tour_packages)}
season_indices = {season: idx for idx, season in enumerate(seasons)}

# Number of tour packages and seasons
n = len(tour_packages)
m = len(seasons)

# Parameters
c_i = data['Cost of Package Per Tourist'].tolist()
c_k = data['Cost per unit of resources'].unique().tolist()
p_i = data['Price'].tolist()
b_r = 1000000  # Maximum budget for resources
R_k = data['#Available Resources'].unique().tolist()
d_min = data['Minimum Demand'].tolist()
d_max = data['Maximum Demand'].tolist()

# Create a pivot table for actual demand
demand_pivot = data.pivot(index='Tour Name', columns='Season', values='Actual Demand')
d_ij = demand_pivot.fillna(0).values.tolist()  # Fill NA values with 0 for missing data

# Initialize r_ijk as a 3-dimensional list
r_ijk = [[[0 for _ in range(len(c_k))] for _ in range(m)] for _ in range(n)]

# Fill r_ijk with values from the dataframe
for row in data.itertuples():
    i = tour_package_indices[row._1]
    j = season_indices[row.Season]
    for k in range(len(c_k)):
        r_ijk[i][j][k] = row._8  # Adjust this index if necessary based on the dataframe structure

m_ij = [50] * m  # Maximum number of packages to offer per season

# Decision Variables
x = pulp.LpVariable.dicts("x", ((i, j) for i in range(n) for j in range(m)), cat='Binary')

# Objective Function
objective = pulp.lpSum(p_i[i] * d_ij[i][j] * x[i, j] for i in range(n) for j in range(m)) - \
            pulp.lpSum(c_i[i] * d_ij[i][j] * x[i, j] for i in range(n) for j in range(m)) - \
            pulp.lpSum(c_k[k] * r_ijk[i][j][k] * x[i, j] for i in range(n) for j in range(m) for k in range(len(c_k)))

# Create the problem
prob = pulp.LpProblem("TourPackageOptimization", pulp.LpMaximize)
prob += objective

# Constraints

# Budget Constraints
for i in range(n):
    for j in range(m):
        prob += pulp.lpSum(c_k[k] * r_ijk[i][j][k] * x[i, j] for k in range(len(c_k))) <= b_r

# Resource Constraints
for k in range(len(c_k)):
    for j in range(m):
        prob += pulp.lpSum(r_ijk[i][j][k] * x[i, j] for i in range(n)) <= R_k[k]

# Demand Constraints
for i in range(n):
    for j in range(m):
        prob += d_ij[i][j] * x[i, j] >= d_min[i]
        prob += d_ij[i][j] * x[i, j] <= d_max[i]

# Package Offering Constraints
for j in range(m):
    prob += pulp.lpSum(x[i, j] for i in range(n)) <= m_ij[j]

# Prevent offering packages in non-existent seasons
for i in range(n):
    for j in range(m):
        if demand_pivot.at[tour_packages[i], seasons[j]] == 0:
            prob += x[i, j] == 0

# Solve the problem
prob.solve()

# Calculate revenue, cost, and profit
total_revenue = 0
total_cost = 0
total_package_cost = 0
total_resource_cost = 0

for i in range(n):
    for j in range(m):
        if pulp.value(x[i, j]) == 1:
            total_revenue += p_i[i] * d_ij[i][j]
            total_package_cost += c_i[i] * d_ij[i][j]
            total_resource_cost += sum(c_k[k] * r_ijk[i][j][k] * d_ij[i][j] for k in range(len(c_k)))

total_cost = total_package_cost + total_resource_cost
profit = total_revenue - total_cost

# Print the results for packages that are offered (x[i, j] == 1)
offered_packages = []
for i in range(n):
    for j in range(m):
        if pulp.value(x[i, j]) == 1:
            offered_packages.append((tour_packages[i], seasons[j]))

# Print the offered packages
for package, season in offered_packages:
    print('---------------------------------------------------')
    print()
    print(f"Package: {package} --> in season: {season} --> is offered.")

# Print the number of packages offered
num_offered_packages = len(offered_packages)
print()
print('---------------------------------------------------')
print(f"Number of packages offered: {num_offered_packages}")
print('---------------------------------------------------')

# Print the objective value as currency
objective_value = int(pulp.value(prob.objective))
print(f"Objective value: {(locale.currency(objective_value, grouping=True))}")



---------------------------------------------------

Package: 5 Days Nile Cruise from Sharm El Sheikh --> in season: Summer --> is offered.
---------------------------------------------------

Package: 13 Days Alexandria, Cairo and Nile Cruise Holidays --> in season: Autumn --> is offered.
---------------------------------------------------

Package: 13 Days Alexandria, Cairo and Nile Cruise Holidays --> in season: Spring --> is offered.
---------------------------------------------------

Package: 14 Days Egypt Adventure Tour --> in season: Autumn --> is offered.
---------------------------------------------------

Package: 14 Days Egypt Adventure Tour --> in season: Spring --> is offered.
---------------------------------------------------

Package: 14 Days Egypt Adventure Tour --> in season: Winter --> is offered.
---------------------------------------------------

Package: 10 Days Egypt & Jordan Tour to Cairo, Nile Cruise & Petra Tour --> in season: Autumn --> is offered.
--------

In [46]:
import pulp
import pandas as pd
import locale

# Set locale for currency formatting
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Load the Excel file
file_path = 'C:/Users/Mohammed/OneDrive/Desktop/use data.xlsx'
data = pd.read_excel(file_path, sheet_name='Data')

# Extract unique tour packages and seasons
tour_packages = data['Tour Name'].unique()
seasons = data['Season'].unique()

# Create integer indices for tour packages and seasons
tour_package_indices = {tour: idx for idx, tour in enumerate(tour_packages)}
season_indices = {season: idx for idx, season in enumerate(seasons)}

# Number of tour packages and seasons
n = len(tour_packages)
m = len(seasons)

# Parameters
c_i = data['Cost of Package Per Tourist'].tolist()
c_k = data['Cost per unit of resources'].unique().tolist()
p_i = data['Price'].tolist()
b_r = 10000000  # Maximum budget for resources
R_k = data['#Available Resources'].unique().tolist()
d_min = data['Minimum Demand'].tolist()
d_max = data['Maximum Demand'].tolist()

# Create a pivot table for actual demand
demand_pivot = data.pivot(index='Tour Name', columns='Season', values='Actual Demand')
d_ij = demand_pivot.fillna(0).values.tolist()  # Fill NA values with 0 for missing data

# Initialize r_ijk as a 3-dimensional list
r_ijk = [[[0 for _ in range(len(c_k))] for _ in range(m)] for _ in range(n)]

# Fill r_ijk with values from the dataframe
for row in data.itertuples():
    i = tour_package_indices[row._1]
    j = season_indices[row.Season]
    for k in range(len(c_k)):
        r_ijk[i][j][k] = row._8  # Adjust this index if necessary based on the dataframe structure

m_ij = [50] * m  # Maximum number of packages to offer per season

# Decision Variables
x = pulp.LpVariable.dicts("x", ((i, j) for i in range(n) for j in range(m)), cat='Binary')

# Objective Function
objective = pulp.lpSum(p_i[i] * d_ij[i][j] * x[i, j] for i in range(n) for j in range(m)) - \
            pulp.lpSum(c_i[i] * d_ij[i][j] * x[i, j] for i in range(n) for j in range(m)) - \
            pulp.lpSum(c_k[k] * r_ijk[i][j][k] * x[i, j] for i in range(n) for j in range(m) for k in range(len(c_k)))

# Create the problem
prob = pulp.LpProblem("TourPackageOptimization", pulp.LpMaximize)
prob += objective

# Constraints

# Budget Constraints
for i in range(n):
    for j in range(m):
        prob += pulp.lpSum(c_k[k] * r_ijk[i][j][k] * x[i, j] for k in range(len(c_k))) <= b_r

# Resource Constraints
for k in range(len(c_k)):
    for j in range(m):
        prob += pulp.lpSum(r_ijk[i][j][k] * x[i, j] for i in range(n)) <= R_k[k]

# Demand Constraints
for i in range(n):
    for j in range(m):
        prob += d_ij[i][j] * x[i, j] >= d_min[i]
        prob += d_ij[i][j] * x[i, j] <= d_max[i]

# Package Offering Constraints
for j in range(m):
    prob += pulp.lpSum(x[i, j] for i in range(n)) <= m_ij[j]

# Prevent offering packages in non-existent seasons
for i in range(n):
    for j in range(m):
        if demand_pivot.at[tour_packages[i], seasons[j]] == 0:
            prob += x[i, j] == 0

# Solve the problem
prob.solve()

# Calculate revenue, cost, and profit
total_revenue = 0
total_cost = 0
total_package_cost = 0
total_resource_cost = 0

for i in range(n):
    for j in range(m):
        if pulp.value(x[i, j]) == 1:
            total_revenue += p_i[i] * d_ij[i][j]
            total_package_cost += c_i[i] * d_ij[i][j]
            total_resource_cost += sum(c_k[k] * r_ijk[i][j][k] * d_ij[i][j] for k in range(len(c_k)))

total_cost = total_package_cost + total_resource_cost
profit = total_revenue - total_cost

# Print the results for packages that are offered (x[i, j] == 1)
offered_packages = []
for i in range(n):
    for j in range(m):
        if pulp.value(x[i, j]) == 1:
            offered_packages.append((tour_packages[i], seasons[j]))

# Print the offered packages
for package, season in offered_packages:
    print('---------------------------------------------------')
    print()
    print(f"Package: {package} --> in season: {season} --> is offered.")

# Print the number of packages offered
num_offered_packages = len(offered_packages)
print()
print('---------------------------------------------------')
print(f"Number of packages offered: {num_offered_packages}")
print('---------------------------------------------------')

# Print the objective value as currency
objective_value = int(pulp.value(prob.objective))
print(f"Objective value: {(locale.currency(objective_value, grouping=True))}")


# Analyze and print reasons for packages not offered
not_offered_packages = []
for i in range(n):
    for j in range(m):
        if pulp.value(x[i, j]) == 0:
            reasons = []
            # Budget constraint
            if sum(c_k[k] * r_ijk[i][j][k] for k in range(len(c_k))) > b_r:
                reasons.append("Exceeds budget")
            # Resource constraint
            for k in range(len(c_k)):
                if sum(r_ijk[i][j][k] for i in range(n)) > R_k[k]:
                    reasons.append("Exceeds available resources")
            # Demand constraints
            if d_ij[i][j] < d_min[i]:
                reasons.append("Below minimum demand")
            if d_ij[i][j] > d_max[i]:
                reasons.append("Above maximum demand")
            # Package offering constraint
            if sum(pulp.value(x[i_, j]) for i_ in range(n)) >= m_ij[j]:
                reasons.append("Maximum packages offered for season")
            # Non-existent seasons
            if demand_pivot.at[tour_packages[i], seasons[j]] == 0:
                reasons.append("Non-existent season")

            not_offered_packages.append((tour_packages[i], seasons[j], reasons))

# Print the reasons for packages not offered
for package, season, reasons in not_offered_packages:
    print('---------------------------------------------------')
    print()
    print(f"Package: {package} --> in season: {season} --> is NOT offered.")
    print("Reasons:")
    for reason in reasons:
        print(f"- {reason}")


---------------------------------------------------

Package: 5 Days Nile Cruise from Sharm El Sheikh --> in season: Summer --> is offered.
---------------------------------------------------

Package: 13 Days Alexandria, Cairo and Nile Cruise Holidays --> in season: Autumn --> is offered.
---------------------------------------------------

Package: 13 Days Alexandria, Cairo and Nile Cruise Holidays --> in season: Spring --> is offered.
---------------------------------------------------

Package: 14 Days Egypt Adventure Tour --> in season: Autumn --> is offered.
---------------------------------------------------

Package: 14 Days Egypt Adventure Tour --> in season: Spring --> is offered.
---------------------------------------------------

Package: 14 Days Egypt Adventure Tour --> in season: Winter --> is offered.
---------------------------------------------------

Package: 10 Days Egypt & Jordan Tour to Cairo, Nile Cruise & Petra Tour --> in season: Autumn --> is offered.
--------